In [7]:
# Download the dataset from Kaggle using the Kaggle API

import opendatasets as od

# Eric's Kaggle API key
# "username":"ericthedataguy",
# "key":"875c0de0faea2fefa081c78eb470a347"

url_cleaned = 'https://www.kaggle.com/datasets/ericthedataguy/cleaned-hotel-review-for-ba820'

od.download(url_cleaned)

Skipping, found downloaded files in "./cleaned-hotel-review-for-ba820" (use force=True to force download)


In [8]:
# Read the dataset into a pandas dataframe

import pandas as pd

filepath = '/Users/hakukazuho/Luxurious-Hotel-Review/cleaned-hotel-review-for-ba820/hotel_reviews_cleaned.csv'
df = pd.read_csv(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [9]:
# Drop outliers in the 'Reviewer_Score' column

# Calculate the IQR
Q1 = df['Reviewer_Score'].quantile(0.25)
Q3 = df['Reviewer_Score'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Drop the outliers and store the results in df_drop
df_drop = df[(df['Reviewer_Score'] >= lower_bound) & (df['Reviewer_Score'] <= upper_bound)]
df_drop.shape[0]


499831

In [10]:
df_drop['Positive_Review'].value_counts(ascending=True).head(10)

Positive_Review
No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam     1
The staff were nice and the room was nicely sized with reasonably priced minibar items                                                                                                                                                                                                                                                                                          

In [11]:
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499831 entries, 1 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               499831 non-null  object 
 1   Additional_Number_of_Scoring                499831 non-null  int64  
 2   Review_Date                                 499831 non-null  object 
 3   Average_Score                               499831 non-null  float64
 4   Hotel_Name                                  499831 non-null  object 
 5   Reviewer_Nationality                        499831 non-null  object 
 6   Negative_Review                             499831 non-null  object 
 7   Review_Total_Negative_Word_Counts           499831 non-null  int64  
 8   Total_Number_of_Reviews                     499831 non-null  int64  
 9   Positive_Review                             499831 non-null  object 
 10  R

In [71]:
# Create subset for reccomendation system 1.0

df2_col_list = ['Hotel_Name', 'Negative_Review', 'Positive_Review', 'Tags']
df2 = df_drop[df2_col_list].iloc[:1000]
df2.head()

,Hotel_Name,Negative_Review,Positive_Review,Tags
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,"[' Leisure trip ', ' Family with young childre..."
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,"[' Leisure trip ', ' Couple ', ' Suite ', ' St..."
5,Hotel Arena,Backyard of the hotel is total mess shouldn t...,Good restaurant with modern design great chil...,"[' Leisure trip ', ' Group ', ' Duplex Double ..."
6,Hotel Arena,Cleaner did not change our sheet and duvet ev...,The room is spacious and bright The hotel is ...,"[' Leisure trip ', ' Group ', ' Duplex Twin Ro..."


In [85]:
df2.to_csv('subset.csv', index=False)

In [72]:
df2['Positive_Review'] = df2['Positive_Review'].str.lower()
df2['Negative_Review'] = df2['Negative_Review'].str.lower()
df2['Tags'] = df2['Tags'].str.lower()
df2['Tags'] = df2['Tags'].str.strip()
df2.head()

,Hotel_Name,Negative_Review,Positive_Review,Tags
1,Hotel Arena,no negative,no real complaints the hotel was great great ...,"[' leisure trip ', ' couple ', ' duplex double..."
2,Hotel Arena,rooms are nice but for elderly a bit difficul...,location was good and staff were ok it is cut...,"[' leisure trip ', ' family with young childre..."
4,Hotel Arena,you when i booked with your company on line y...,amazing location and building romantic setting,"[' leisure trip ', ' couple ', ' suite ', ' st..."
5,Hotel Arena,backyard of the hotel is total mess shouldn t...,good restaurant with modern design great chil...,"[' leisure trip ', ' group ', ' duplex double ..."
6,Hotel Arena,cleaner did not change our sheet and duvet ev...,the room is spacious and bright the hotel is ...,"[' leisure trip ', ' group ', ' duplex twin ro..."


In [73]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# positive_review = df2['Positive_Review']
# word_tokenize(positive_review)
df2['positive_tokenized'] = df2['Positive_Review'].apply(lambda x: word_tokenize(x))
df2['negative_tokenized'] = df2['Negative_Review'].apply(lambda x: word_tokenize(x))

stop_words = set(stopwords.words('english'))
df2['positive_filtered'] = df2['positive_tokenized'].apply(lambda x: [word for word in x if word not in stop_words])
df2['negative_filtered'] = df2['negative_tokenized'].apply(lambda x: [word for word in x if word not in stop_words])

df2.head()

,Hotel_Name,Negative_Review,Positive_Review,Tags,positive_tokenized,negative_tokenized,positive_filtered,negative_filtered
1,Hotel Arena,no negative,no real complaints the hotel was great great ...,"[' leisure trip ', ' couple ', ' duplex double...","[no, real, complaints, the, hotel, was, great,...","[no, negative]","[real, complaints, hotel, great, great, locati...",[negative]
2,Hotel Arena,rooms are nice but for elderly a bit difficul...,location was good and staff were ok it is cut...,"[' leisure trip ', ' family with young childre...","[location, was, good, and, staff, were, ok, it...","[rooms, are, nice, but, for, elderly, a, bit, ...","[location, good, staff, ok, cute, hotel, break...","[rooms, nice, elderly, bit, difficult, rooms, ..."
4,Hotel Arena,you when i booked with your company on line y...,amazing location and building romantic setting,"[' leisure trip ', ' couple ', ' suite ', ' st...","[amazing, location, and, building, romantic, s...","[you, when, i, booked, with, your, company, on...","[amazing, location, building, romantic, setting]","[booked, company, line, showed, pictures, room..."
5,Hotel Arena,backyard of the hotel is total mess shouldn t...,good restaurant with modern design great chil...,"[' leisure trip ', ' group ', ' duplex double ...","[good, restaurant, with, modern, design, great...","[backyard, of, the, hotel, is, total, mess, sh...","[good, restaurant, modern, design, great, chil...","[backyard, hotel, total, mess, happen, hotel, ..."
6,Hotel Arena,cleaner did not change our sheet and duvet ev...,the room is spacious and bright the hotel is ...,"[' leisure trip ', ' group ', ' duplex twin ro...","[the, room, is, spacious, and, bright, the, ho...","[cleaner, did, not, change, our, sheet, and, d...","[room, spacious, bright, hotel, located, quiet...","[cleaner, change, sheet, duvet, everyday, made..."


In [76]:
import nltk

# Download the required NLTK resources
nltk.download('averaged_perceptron_tagger')

# Function to extract nouns from a list of tagged words
def extract_nouns(tagged_words):
    nouns = []
    for word, tag in tagged_words:
        if tag.startswith('N'):
            nouns.append(word)
    return nouns

# Apply POS tagging and extract nouns for positive_filtered column
df2['positive_noun'] = df2['positive_filtered'].apply(lambda x: extract_nouns(nltk.pos_tag(x)))

# Apply POS tagging and extract nouns for negative_filtered column
df2['negative_noun'] = df2['negative_filtered'].apply(lambda x: extract_nouns(nltk.pos_tag(x)))

# Remove duplicate words from positive_noun and negative_noun columns
df2['positive_noun'] = df2['positive_noun'].apply(lambda x: list(set(x)))
df2['negative_noun'] = df2['negative_noun'].apply(lambda x: list(set(x)))
df2.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hakukazuho/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Hotel_Name,Negative_Review,Positive_Review,Tags,positive_tokenized,negative_tokenized,positive_filtered,negative_filtered,positive_noun,negative_noun
1,Hotel Arena,no negative,no real complaints the hotel was great great ...,"[' leisure trip ', ' couple ', ' duplex double...","[no, real, complaints, the, hotel, was, great,...","[no, negative]","[real, complaints, hotel, great, great, locati...",[negative],"[quality, payments, option, location, check, c...",[]
2,Hotel Arena,rooms are nice but for elderly a bit difficul...,location was good and staff were ok it is cut...,"[' leisure trip ', ' family with young childre...","[location, was, good, and, staff, were, ok, it...","[rooms, are, nice, but, for, elderly, a, bit, ...","[location, good, staff, ok, cute, hotel, break...","[rooms, nice, elderly, bit, difficult, rooms, ...","[breakfast, staff, hotel, range, location]","[steps, level, coffee, fridge, story, boiler, ..."
4,Hotel Arena,you when i booked with your company on line y...,amazing location and building romantic setting,"[' leisure trip ', ' couple ', ' suite ', ' st...","[amazing, location, and, building, romantic, s...","[you, when, i, booked, with, your, company, on...","[amazing, location, building, romantic, setting]","[booked, company, line, showed, pictures, room...","[setting, location, building]","[please, company, pictures, told, pricing, pho..."
5,Hotel Arena,backyard of the hotel is total mess shouldn t...,good restaurant with modern design great chil...,"[' leisure trip ', ' group ', ' duplex double ...","[good, restaurant, with, modern, design, great...","[backyard, of, the, hotel, is, total, mess, sh...","[good, restaurant, modern, design, great, chil...","[backyard, hotel, total, mess, happen, hotel, ...","[stairs, restaurant, design, park, hotel, place]","[stars, mess, hotel]"
6,Hotel Arena,cleaner did not change our sheet and duvet ev...,the room is spacious and bright the hotel is ...,"[' leisure trip ', ' group ', ' duplex twin ro...","[the, room, is, spacious, and, bright, the, ho...","[cleaner, did, not, change, our, sheet, and, d...","[room, spacious, bright, hotel, located, quiet...","[cleaner, change, sheet, duvet, everyday, made...","[park, room, hotel, beautiful]","[change, duvet, sheet, body, gel, floor, bed, ..."


In [83]:
df3 = df2.groupby('Hotel_Name').agg({'positive_noun': 'sum', 'negative_noun': 'sum'}).reset_index()
df3.rename(columns={'positive_noun': 'positive_tags', 'negative_noun': 'negative_tags'}, inplace=True)

df3['positive_tags'] = df3['positive_tags'].apply(lambda x: list(x))
df3['negative_tags'] = df3['negative_tags'].apply(lambda x: list(x))
df3


,Hotel_Name,positive_tags,negative_tags
0,Apex Temple Court Hotel,"[requests, choices, check, attractions, journe...","[itineraries, reception, locate, toilet, conta..."
1,Hotel Arena,"[quality, payments, option, location, check, c...","[steps, level, coffee, fridge, story, boiler, ..."
2,K K Hotel George,"[facilities, delight, staff, bathroom, shower,...","[nothing, hotel, lobby, years, lug, access, co..."


In [84]:
from collections import Counter

# Function to get the top 10 most frequently mentioned tags
def get_top_tags(tags):
    tag_counts = Counter(tags)
    top_tags = [tag for tag, count in tag_counts.most_common(10)]
    return top_tags

# Get the top 10 most frequently mentioned tags for positive_noun and negative_noun columns
df3['top_positive'] = df3['positive_tags'].apply(lambda x: get_top_tags(x))
df3['top_negative'] = df3['negative_tags'].apply(lambda x: get_top_tags(x))

df3.head()


,Hotel_Name,positive_tags,negative_tags,top_positive,top_negative
0,Apex Temple Court Hotel,"[requests, choices, check, attractions, journe...","[itineraries, reception, locate, toilet, conta...","[room, location, hotel, bathroom, staff, bed, ...","[room, hotel, breakfast, food, service, staff,..."
1,Hotel Arena,"[quality, payments, option, location, check, c...","[steps, level, coffee, fridge, story, boiler, ...","[staff, hotel, room, location, rooms, bed, par...","[room, hotel, bathroom, rooms, construction, w..."
2,K K Hotel George,"[facilities, delight, staff, bathroom, shower,...","[nothing, hotel, lobby, years, lug, access, co...","[location, staff, breakfast, hotel, room, cour...","[room, breakfast, nothing, bathroom, hotel, ro..."


### Some observation

* There are both positive & negative feedback in some single positive/negative reviews. Might need to pool both positive & negative reviews together, conduct sentiment analysis, then decide the keywords extracted goes to positive or negative tag pool.

* 

### What's Next?

1. Tags and Review have different format

2. Remove redundant words using stop words

3. Extract keywords from Reviews

4. Store keywords to negative/positive tag pools

5. Try ngrams using CountVectorizer()

6. Should we assign more recent reviews with higher weights? How?

In [15]:
# Remove reduntant words from the Negative_Review and Positive_Review columns

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

positive = df2['Positive_Review']
negative = df2['Negative_Review']

# Filter out the stop words
filtered_positive = [word for word in positive if word not in stop_words]
filtered_negative = [word for word in negative if word not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hakukazuho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: unhashable type: 'list'